# 基礎設置
`langchain_setup` 包含了使用 langchain 開發所需的環境和我寫的一些工具。

透過 import 這個套件，會自動讀取 `~/.cache/.env` 或 `langcahin_setup/.env` 中，設定好環境變數，並做好網路連接的設定。

In [1]:
import langchain_setup

另外這個套件可以根據環境變數 `OPENAI_API_TYPE` 的有無或其值來自動切換使用`OpenAI` 和 `AzureOpenAI`，並且依環境變數 `DEFAULT_...MODEL` 來自動設定模型名稱。讓我們私人或公用時都可以用同一份程式碼不用改。

In [2]:
from langchain_setup import OpenAI
model = OpenAI()
print(type(model))
print(model.model_name)
print(model.deployment_name)

<class 'langchain.llms.openai.AzureOpenAI'>
text-davinci-003
vin-test


# 計算成本
Langchain 有提供一個 context manager `get_openai_callback`。在這個 context manager 的範圍內，其會插入一個 callback 計算使用的 token 數。

In [3]:
from langchain_setup import ChatOpenAI
from langchain.callbacks import get_openai_callback

model = ChatOpenAI(n=2)

with get_openai_callback() as cb:
    result = model.predict("教練，我想學飛天御劍流。")
    print(cb)

Tokens Used: 1340
	Prompt Tokens: 27
	Completion Tokens: 1313
Successful Requests: 1
Total Cost (USD): $0.0026665


其設有一個預先寫死的價目表 (USD per one thousand tokens)，而我們可以手動調整這個價目表。

In [4]:
from langchain.callbacks.openai_info import MODEL_COST_PER_1K_TOKENS
from pprint import pprint

pprint(MODEL_COST_PER_1K_TOKENS)

{'ada': 0.0004,
 'ada-finetuned-legacy': 0.0016,
 'babbage': 0.0005,
 'babbage-002-azure-finetuned': 0.0004,
 'babbage-002-azure-finetuned-completion': 0.0004,
 'babbage-002-finetuned': 0.0016,
 'babbage-002-finetuned-completion': 0.0016,
 'babbage-finetuned-legacy': 0.0024,
 'code-davinci-002': 0.02,
 'curie': 0.002,
 'curie-finetuned-legacy': 0.012,
 'davinci-002-azure-finetuned': 0.002,
 'davinci-002-azure-finetuned-completion': 0.002,
 'davinci-002-finetuned': 0.012,
 'davinci-002-finetuned-completion': 0.012,
 'davinci-finetuned-legacy': 0.12,
 'gpt-3.5-turbo': 0.0015,
 'gpt-3.5-turbo-0301': 0.0015,
 'gpt-3.5-turbo-0301-completion': 0.002,
 'gpt-3.5-turbo-0613': 0.0015,
 'gpt-3.5-turbo-0613-completion': 0.002,
 'gpt-3.5-turbo-0613-finetuned': 0.012,
 'gpt-3.5-turbo-0613-finetuned-completion': 0.016,
 'gpt-3.5-turbo-16k': 0.003,
 'gpt-3.5-turbo-16k-0613': 0.003,
 'gpt-3.5-turbo-16k-0613-completion': 0.004,
 'gpt-3.5-turbo-16k-completion': 0.004,
 'gpt-3.5-turbo-completion': 0.002,


In [5]:
MODEL_COST_PER_1K_TOKENS[model.model_name] = 0.0001
with get_openai_callback() as cb:
    result = model.predict("教練，我想學飛天御劍流。")
    print(cb)

Tokens Used: 1433
	Prompt Tokens: 27
	Completion Tokens: 1406
Successful Requests: 1
Total Cost (USD): $0.0028525


# Cache

在某些情況，我們會發現自己重複在用同樣的 input 做同樣生成 (generation)，這時候我們就可以用到快取 (caching) 來節省我們的成本。

譬如下面舉個例子，要對兩篇很長的文章做一個摘要，這兩篇長到塞不下一個 prompt (超過模型可處理的方法)，於是我們想出一個辦法，先對這兩篇文章各自先產出一個摘要，再總結這兩個摘要成為最終的摘要。在下面這個例子中，針對文章個別產出摘要的第一階段可能就會有重複做的問題。

In [6]:
import langchain
from langchain_setup import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.cache import InMemoryCache, SQLiteCache

langchain.llm_cache = InMemoryCache()

cache_llm = ChatOpenAI(cache=True)
non_cache_llm = ChatOpenAI(cache=False) # default

texts = [
    "術式順轉「蒼」：強化版無下限術式，能夠從「正無限」變換為「負無限」，用強勁的吸引力讓物體和物體互相吸收產生碰撞，也可以用這招來實現瞬間移動。",
    "虛式「茈」：結合術式順轉「蒼」和術式反轉「赫」，讓正負的無限之力進行衝突，再將其中產生的假想質量發射出去，力量強大到像是黑洞般能夠吞噬一切。"
]
prompt_template1 = "請將文章「{text}」摘要成一句話"
prompt_template2 = "請將摘要們「{text}」進一步摘要成一個摘要"

with get_openai_callback() as cb:
    summary_1 = cache_llm.predict(prompt_template1.format(text=texts[0]))
    summary_2 = cache_llm.predict(prompt_template1.format(text=texts[1]))

    summary_of_summary_1 = non_cache_llm.predict(prompt_template2.format(text=summary_1 + '\n' + summary_2))
    print(summary_of_summary_1)
    print(cb)

print("===============================================================================")

new_prompt_template2 = "請將摘要們「{text}」以正統物理學的觀點用一句話做出總結"

with get_openai_callback() as cb:
    summary_1 = cache_llm.predict(prompt_template1.format(text=texts[0]))
    summary_2 = cache_llm.predict(prompt_template1.format(text=texts[1]))

    summary_of_summary_2 = non_cache_llm.predict(new_prompt_template2.format(text=summary_1 + '\n' + summary_2))
    print(summary_of_summary_2)
    print(cb)

摘要：這個摘要描述了一種強化版的術式「蒼」，它能夠從正無限轉變為負無限，並利用強勁的吸引力引起物體之間的碰撞，同時實現瞬間移動。通過結合術式順轉「蒼」和術式反轉「赫」，虛式「茈」能夠發射出具有假想質量和黑洞般吞噬力量的能量。
Tokens Used: 743
	Prompt Tokens: 431
	Completion Tokens: 312
Successful Requests: 3
Total Cost (USD): $0.0012705
這些描述涉及超越現有物理學理論的概念，無法用正統物理學的觀點做出總結。
Tokens Used: 245
	Prompt Tokens: 194
	Completion Tokens: 51
Successful Requests: 1
Total Cost (USD): $0.000393


將文件初步摘要的步驟沒有變，只有將產生的摘要進行再摘要的 prompt 變了，因此可以看到第二次執行時，只有一個做再摘要的呼叫 (request)，其它則是從快取 (cache) 取用。

而其快取 (cache) 內部其實就是將該呼叫 (request) 的 input prompt, 模型, 設定等等作為一個 key，輸出的生成內容作為 value，的一個大型 `dict`

這邊還有另外一個點在於，如果你是想要每次的生成內容有點隨機性 (temperature>0)，caching 會直接無視你想增加隨機性的設定，

In [7]:
pprint(langchain.llm_cache._cache)

{('[{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "\\u8acb\\u5c07\\u6587\\u7ae0\\u300c\\u865b\\u5f0f\\u300c\\u8308\\u300d\\uff1a\\u7d50\\u5408\\u8853\\u5f0f\\u9806\\u8f49\\u300c\\u84bc\\u300d\\u548c\\u8853\\u5f0f\\u53cd\\u8f49\\u300c\\u8d6b\\u300d\\uff0c\\u8b93\\u6b63\\u8ca0\\u7684\\u7121\\u9650\\u4e4b\\u529b\\u9032\\u884c\\u885d\\u7a81\\uff0c\\u518d\\u5c07\\u5176\\u4e2d\\u7522\\u751f\\u7684\\u5047\\u60f3\\u8cea\\u91cf\\u767c\\u5c04\\u51fa\\u53bb\\uff0c\\u529b\\u91cf\\u5f37\\u5927\\u5230\\u50cf\\u662f\\u9ed1\\u6d1e\\u822c\\u80fd\\u5920\\u541e\\u566c\\u4e00\\u5207\\u3002\\u300d\\u6458\\u8981\\u6210\\u4e00\\u53e5\\u8a71"}}]', '{"lc": 1, "type": "constructor", "id": ["langchain", "chat_models", "azure_openai", "AzureChatOpenAI"], "kwargs": {"deployment_name": "gpt-turbo-test-20230330", "cache": true, "openai_api_type": "azure", "openai_api_version": "2023-07-01-preview", "openai_api_base": "https://oai-it-test.privatelink